In [1]:
import numpy as np
import random
from datetime import datetime
import time
from collections import defaultdict
from helper_func import *
import pickle
import multiprocessing
import keras
from multiprocessing import Pool
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from collections import deque
from keras.activations import relu, linear
from keras.losses import mean_squared_error
from keras.optimizers import Adam
import tensorflow as tf

%load_ext autoreload

Using TensorFlow backend.
/Users/jgaustad/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jgaustad/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jgaustad/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jgaustad/anaconda3/lib/python3.7/site-pa

In [2]:
# The inputs are 28x28x28 volumes with a single channel, and the  
# batch size is 4  
input_shape =(4, 28, 28, 28, 1)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv3D(2, 3, activation='relu', input_shape=input_shape[1:])(x)
print(y)
print(y.shape)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("conv3d/Relu:0", shape=(4, 26, 26, 26, 2), dtype=float32)
(4, 26, 26, 26, 2)


In [3]:
%autoreload 2

In [4]:
class pentago:
    """
    
    """

    def __init__(self, state = None):
        """Initializes the class reservation"""
        #print('initializing')
        
        if state == None:
            self.state = state = np.zeros((6,6), dtype=np.int)
        self.history = []
        self.winner = None
        self.gameover = False
        self.player_turn = 1
    
    def current_board_state(self):
        # need to return a copy or bad stuff happens
        return copy.copy(self.state)
    
    def game_history(self, player, move, cuad, rotatation):
        self.history.append((boardstate_to_ideal_key(self.state), ideal_state(self.state), player, move, cuad, rotatation))
        #return self.history

    def find_winner(self, board_state):
        player1_win = False
        player_min1_win = False
        diagonal1 = board_state.diagonal()
        diagonal2 = np.fliplr(board_state).diagonal()
        winning_slices =  np.vstack([board_state[1:,:].T, board_state[:-1,:].T, # all columns
                              board_state[:,1:], board_state[:,:-1], # all rows
                              diagonal1[1:], diagonal1[:-1], # diagonal 1
                              diagonal2[1:],diagonal2[1:], # diagonal 2
                              board_state.diagonal(offset=1), board_state.diagonal(offset=-1), # diagonal offsets 
                              np.fliplr(board_state).diagonal(offset=1), np.fliplr(board_state).diagonal(offset=-1)] ) # diagonal offsets
        sums = np.dot(winning_slices, np.array([1,1,1,1,1]))
        if 5 in sums: player1_win = True
        if -5 in sums: player_min1_win = True
        if player1_win == True or player_min1_win == True:
           # print("Player 1 winner?", player1_win, "Player -1 winner?", player_min1_win)
            self.gameover = True
            if player1_win == True:
                self.winner = 1
            elif player_min1_win ==True:
                self.winner = -1
            self.history.append(self.winner)
        return "Win"

    def check_gameover(self):
        if not 0 in self.state:
              self.gameover = True
              #print("The game board is full!")
        
    def full_move(self, move, cuad, direction, player, dtype=np.int):
        if player != self.player_turn:
            print( "error, wrong player turn. No move taken.")
            return 'Error, wrong player turn.'
        self.state = fullmove(self.state,move, cuad, direction, player)


        self.game_history(move, player, cuad, direction)
        self.find_winner(self.state) #return in find_winner if a winner is found
        self.check_gameover() #return in check_gameover
        if player == 1:
            self.player_turn = -1
        else:
            self.player_turn = 1
        #print('Successful Move')
        return self.state



In [5]:
class q_table:

    def __init__(self,length=0, games_played=0):
        """Initializes the class reservation"""
        self.time = datetime.now()
        self.length = length
        self.q_dict = {}
        self.games_played = games_played

  #def time(self):
    #self.time = time

    def length(self):
        self.length += 1
    #self.length = length  
    
    def get_q_value(self, boardstate):
        return self.q_dict.get(boardstate, (0, 0))
    
    def update_q_value(self, boardstate, new_val, update_function = None):
        q_val, n = self.get_q_value(boardstate) 
        if update_function:
            #print('using custom function')
            self.q_dict[boardstate] = update_function(q_val, n, new_val)
        else:
            self.q_dict[boardstate] = [new_val, n+1]
        return self.q_dict[boardstate]
    
    def update_post_game(self, history, update_fn):
        winner = history[-1]
        
        for boardposition in history[-2::-1]:
            key = boardposition[0]
            #print(key, winner)
            self.update_q_value(key, winner, update_fn)

    def update_post_game2(self, history, update_fn, decay_reward = .9):
        winner = history[-1]
        
        for boardposition in history[-2::-1]:
            key = boardposition[0]
            #print(key, winner)
            self.update_q_value(key, winner, update_fn)
            winner *= decay_reward

    
    

In [6]:
class cnn_model:

    def __init__(self,list_density=None,lr=0.02):
        self.model = self.build_model(list_density,lr)

    def build_model(self,den,lr):
        model = Sequential()
        #model.add(tf.keras.Input(shape=(6,6,2,)))
        #for layer in den:
        model.add(Conv2D(
                            filters = 64, 
                            kernel_size = (3,3),
                            strides = (3,3),
                            padding = 'valid',
                            activation = relu,
                            input_shape = (6,6,2)))
        model.add(Flatten())
        model.add(Dense(256, activation = relu))
        model.add(Dense(128, activation = relu))


        model.add(Dense(1, activation=linear))
        
        model.compile(loss=mean_squared_error,optimizer=Adam(lr=lr))
        print(model.summary())
        return model



    def update_model(self,states_batch, q_batch, epochs = 1):

        self.model.fit(states_batch,q_batch,epochs = epochs, verbose=1)

    def predict_model(self, state_batch):
        return self.model.predict_on_batch(state_batch)
        

In [7]:
c = cnn_model()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
x = np.array([[1,1,1,-1,1,1],[0,0,0,0,0,0],[0,0,0,1,1,0],[0,0,1,-1,-1,0],[0,0,1,-1,1,0],[-1,-1,1,0,0,0]])
x = boardstate_to_cnn_input(x)
print(x.shape)
out= c.model.predict(x.reshape(1,6,6,2))
print(out.shape)
out

(6, 6, 2)


(1, 1)


array([[-0.05547169]], dtype=float32)

In [35]:
class nn_agent:
    
    def __init__(self, nn, player = 1, epsilon = 1, epsilon_decay = .99995, epsilon_min = .5):
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.nn = nn
        self.player = player
        self.epsilon_min = epsilon_min
        
            
    def get_avail_moves(self,boardstate):
        """
        This method creates a list with available spaces in the board and combination of quadrant and rotation
        The input is the board state (6x6) numpy array
        """
        x = np.where(boardstate == 0)
        #print(x)
        available_positions_for_placement = list(zip(x[0], x[1]))
        
        # all available positions (p), quadrants(q), rotations(r)
        available_moves = [(p,q,r) for p in available_positions_for_placement for q in [1,2,3,4] for r in [-1,1]]
        #print(len(available_moves))
        return available_moves
    
    def get_possible_next_boardstates(self, boardstate):
        next_possible_boardstates = defaultdict(list)
        for move in self.get_avail_moves(boardstate):
            possible_boardstate = fullmove(boardstate,*move, self.player)
            #print(possible_boardstate)
            key = boardstate_to_ideal_key(possible_boardstate)
            cnn_input = boardstate_to_cnn_input(possible_boardstate)
            #print(cnn_input)
            #print(key)
            next_possible_boardstates[key].append((cnn_input, move))
            
        return next_possible_boardstates
    
    def make_move(self, game):
        
        # get the current boardstate from the pentago class
        boardstate = game.current_board_state()
        
        # get possible next possible boardstates
        t0 = time.time()
        next_possible_boardstates = self.get_possible_next_boardstates(boardstate)
        key_list = list(next_possible_boardstates.keys())
        #print("get possible boardstates", time.time()-t0)
        # determine if to take random move
        if np.random.rand() < self.epsilon:
            random_bs = random.choice(key_list)
            random_mv = next_possible_boardstates[random_bs][0][1] # capture the move needed
            
            game.full_move(*random_mv,self.player)
            
        else:
            #print("not random", self.player)
           # t0 = time.time()
            nn_input_boardstate = []
            move = []
            for k,v in next_possible_boardstates.items():
                nn_input_boardstate.append(v[0][0])
                #print(v[0][0].shape)
                move.append(v[0][1])
          
            #print(nn_input_boardstate, type(nn_input_boardstate))
            #nn_input_batch = np.concatenate(nn_input_boardstate)
            nn_input_batch = np.array(nn_input_boardstate)
            #print(nn_input_batch, type(nn_input_batch))
            #print(nn_input_boardstate)
            q_values = self.nn.predict_model(nn_input_batch)
            #print("get q_values boardstates", time.time()-t0)

            #print(q_values, type(q_values))
            #print("----")
            q_values *= self.player
            #q_values_list = [x*self.player for x in list(q_values)]
            #print(q_values_list)
            # get random index of a max value
            #print('length fo q_values_list = ', len(q_values_list))
            max_q = np.argmax(q_values)
            #index_of_all_max = [i for i in range(len(q_values)) if q_values[i] == max_q]
            #random_max_q_index = random.choice(index_of_all_max)
            #print("MAX VALUE:", q_values_list[random_max_q_index])
            mv_to_take = move[max_q]
            game.full_move(*mv_to_take, self.player)
        
        #if self.epsilon > self.epsilon_min:
        #    self.epsilon *= self.epsilon_decay 
        #else:
        #    self.epsilon = self.epsilon_min

In [36]:
class nn_agent_multigame:
    
    def __init__(self, nn, player = 1, epsilon = 1, epsilon_decay = .99995, epsilon_min = .5):
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.nn = nn
        self.player = player
        self.epsilon_min = epsilon_min
        
            
    def get_avail_moves(self,boardstate):
        """
        This method creates a list with available spaces in the board and combination of quadrant and rotation
        The input is the board state (6x6) numpy array
        """
        x = np.where(boardstate == 0)
        #print(x)
        available_positions_for_placement = list(zip(x[0], x[1]))
        
        # all available positions (p), quadrants(q), rotations(r)
        available_moves = [(p,q,r) for p in available_positions_for_placement for q in [1,2,3,4] for r in [-1,1]]
        #print(len(available_moves))
        return available_moves
    
    def get_possible_next_boardstates(self, boardstate):
        next_possible_boardstates = defaultdict(list)
        for move in self.get_avail_moves(boardstate):
            possible_boardstate = fullmove(boardstate,*move, self.player)
            #print(possible_boardstate)
            key = boardstate_to_ideal_key(possible_boardstate)
            cnn_input = boardstate_to_cnn_input(possible_boardstate)
            #print(cnn_input)
            #print(key)
            next_possible_boardstates[key].append((cnn_input, move))
            
        return next_possible_boardstates
    
    def make_move(self, game):
        
        # get the current boardstate from the pentago class
        boardstate = game.current_board_state()
        
        # get possible next possible boardstates
        next_possible_boardstates = self.get_possible_next_boardstates(boardstate)
        key_list = list(next_possible_boardstates.keys())
        
        # determine if to take random move
        if np.random.rand() < self.epsilon:
            random_bs = random.choice(key_list)
            random_mv = next_possible_boardstates[random_bs][0][1] # capture the move needed
            
            game.full_move(*random_mv,self.player)
            
        else:
            #print("not random", self.player)
            nn_input_boardstate = []
            move = []
            for k,v in next_possible_boardstates.items():
                nn_input_boardstate.append(v[0][0])
                #print(v[0][0].shape)
                move.append(v[0][1])
            
            
            #print(nn_input_boardstate, type(nn_input_boardstate))
            #nn_input_batch = np.concatenate(nn_input_boardstate)
            nn_input_batch = np.array(nn_input_boardstate)
            #print(nn_input_batch, type(nn_input_batch))
            #print(nn_input_boardstate)
            q_values = self.nn.predict_model(nn_input_batch)
            #print(q_values, type(q_values))
            #print("----")
            q_values *= self.player
            #q_values_list = [x*self.player for x in list(q_values)]
            #print(q_values_list)
            # get random index of a max value
            #print('length fo q_values_list = ', len(q_values_list))
            max_q = np.argmax(q_values)
            #index_of_all_max = [i for i in range(len(q_values)) if q_values[i] == max_q]
            #random_max_q_index = random.choice(index_of_all_max)
            #print("MAX VALUE:", q_values_list[random_max_q_index])
            mv_to_take = move[max_q]
            game.full_move(*mv_to_take, self.player)
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay 
        else:
            self.epsilon = self.epsilon_min

In [37]:
x = np.array([1,2,3,4,2,1,4])
np.argmax(x)

3

In [38]:
nn_1 = cnn_model([72,144])

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_6 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_17 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None


In [91]:
agent1 = nn_agent(player = 1,  nn = nn_1, epsilon_min = 0, epsilon_decay = .999, epsilon = 0)
agent2 = nn_agent(player = -1,  nn = nn_2, epsilon_min = 0, epsilon = 0)

In [93]:
t0 = time.time()
g = pentago()
while g.gameover == False:
    agent1.make_move(g)
    if g.gameover == True: break
    agent2.make_move(g)
    #break

#bs = agent1.get_possible_next_boardstates(g.current_board_state())
    
print(time.time()-t0)
print(len(g.history))
g.history

1.9728119373321533
28


[('[000000000000000000000000000000000100]',
  array([[0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0]]),
  (0, 0),
  1,
  1,
  -1),
 ('[000000000000000000000100000000000200]',
  array([[ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  1,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  0]]),
  (0, 0),
  -1,
  1,
  -1),
 ('[000000000000000000000201000000000100]',
  array([[ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, -1,  0,  1],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  1,  0,  0]]),
  (0, 0),
  1,
  1,
  -1),
 ('[000000000000000000000102000000000201]',
  array([[ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  1,  0, -1],
  

In [29]:
g.history[:2]
def reward_func(history, decay_factor = .9):
    winner = history[-1]
    nn_inputs = []
    rewards = []
    for boardposition in history[-2::-1]:
        nn_inputs.append(boardposition[1])
        rewards.append(winner)
        winner *= decay_factor
    return nn_inputs, rewards
a = reward_func(g.history)
print(type(a[0]), type(a[1]), len(a[0]), len(a[1]))
x = np.array([boardstate_to_cnn_input(bs) for bs in a[0]])
Y = np.array(a[1])
print(x.shape, Y.shape)

nn_1.update_model(x,Y)

<class 'list'> <class 'list'> 28 28
(28, 6, 6, 2) (28,)
Epoch 1/1
28/28 [==============================] - 1s 34ms/step - loss: 0.1720


In [30]:
def little_sim(agents):
    agent1, agent2 = agents
    g = pentago()
    while g.gameover == False:
        agent1.make_move(g)
        if g.gameover ==True: break
        agent2.make_move(g)
    #print('gameover.')
    return g
#little_sim((agent1,agent2))
#if __name__ == '__main__':
#    with Pool(1) as p:
#        game_returns = p.map(little_sim, [(agent1,agent2)]*12)

In [31]:
def test(x):
    return x, str(time.time())[-1], x*time.time()

if __name__ == '__main__':
    with Pool(6) as p:
        returns = p.map(test, [x for x in range(10)])
print(returns)

[(0, '2', 0.0), (1, '9', 1607207032.787425), (2, '2', 3214414065.575404), (3, '4', 4821621098.363877), (4, '6', 6428828131.152436), (5, '4', 8036035163.939341), (6, '8', 9643242196.727467), (7, '9', 11250449229.516958), (8, '1', 12857656262.305271), (9, '6', 14464863295.093979)]


In [96]:
def big_sim_parallel_nn(agent1, agent2, n_steps = 1, games_per_step = 32, nn_to_update = [], parallel_threads = 6):
    game_times = []
    evals = []
    eval_times = []
    epsilons = []
    nn_update_times = []
    winner_list = []
    
    for n in range(n_steps):
        print('game_step', n, end = ' ')
        game_start = time.time()
        
        #if __name__ == '__main__':
        #    with Pool(parallel_threads) as p:
        #        game_returns = p.map(little_sim, [(agent1,agent2)]*games_per_step)
        game_returns = [little_sim((agent1,agent2)) for x in range(games_per_step)] #comment our parallelilization if needed.
        
        #games_in_progress = [pentago() for g in games_per_step]
        #finished_games = []
        #while 
            
        game_times.append(time.time()-game_start)
            
        player1_winner = 0
        player2_winner = 0
        # check for winner and create update batch
        nn_input_batch = []
        rewards_for_batch = []
        for game in game_returns:
            if game.winner:
                if game.winner == 1: player1_winner += 1
                else: player2_winner += 1
                
                # accumulate rewards and inputs for training
                boardstates, rewards = reward_func(game.history)
                nn_input_batch += [boardstate_to_cnn_input(bs) for bs in boardstates] # add nn_inputs to training list (x)
                rewards_for_batch += rewards # add rewards to training list (Y)
        
        # evaluate the model
        t0 = time.time()
        for nn in nn_to_update:
            evals.append(nn.model.evaluate(np.array(nn_input_batch), np.array(rewards_for_batch)))
        eval_times.append(time.time() - t0)

        
        # train the neural network
        t0 = time.time()
        print(f"updating with {len(nn_input_batch)}, {len(rewards_for_batch)} training batch.")
        for nn in nn_to_update:
            nn.update_model(np.array(nn_input_batch), np.array(rewards_for_batch))
        nn_update_times.append(time.time() - t0)


        print("player 1 wins:", player1_winner)
        print("player 2 wins:", player2_winner)
        print("simulations took", game_times[-1], "seconds.")
        print("neural network update time:", nn_update_times[-1], "seconds.")
        print("eval score on batch:", evals[-1])
        print("epsilons agent1 and agent2:", agent1.epsilon, agent2.epsilon)
        epsilons.append((agent1.epsilon, agent2.epsilon))
        if agent1.epsilon > agent1.epsilon_min: agent1.epsilon *= agent1.epsilon_decay
        else: agent1.epsilon == agent1.epsilon_min
        if agent2.epsilon > agent2.epsilon_min: agent2.epsilon *= agent2.epsilon_decay
        else: agent2.epsilon == agent2.epsilon_min

    # end of simulation runs, save q_table(s) to disk
    nn_num = 1
    time_str = str(datetime.now())[:19].replace(':','_')
    for nn in nn_to_update:
        with open(f'CNN_{nn_num}_'+time_str+'.pickle', 'wb') as file:
            pickle.dump(nn, file, protocol = pickle.HIGHEST_PROTOCOL)
        nn_num += 1
    
    return epsilons, nn_update_times, player1_winner, player2_winner, evals, eval_times
            

In [73]:
# Note you will overwrite this q_table and agents if you run this cell again.    Verify you won't lose your data!
with open('decay_q_table1_2020-11-29 12_09_00.pickle', 'rb') as file:
    qtable1 =  pickle.load(file)
nn_1 = cnn_model()
agent1 = nn_agent(player = 1,  nn = nn_1, epsilon_min = .02, epsilon = 1, epsilon_decay = .99)
agent2 = nn_agent(player = -1,  nn = nn_1, epsilon_min = .15, epsilon = 1, epsilon_decay = .995)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_15 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_44 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None


In [74]:
big_sim_parallel_nn(agent1, agent2, n_steps=1, games_per_step=32, nn_to_update=[nn_1], parallel_threads=6)

797/797 [==============================] - 1s 974us/step
updating with 797, 797 training batch.
Epoch 1/1
797/797 [==============================] - 1s 2ms/step - loss: 0.3802
player 1 wins: 16
player 2 wins: 13
simulations took 56.79386496543884 seconds.
neural network update time: 1.7375359535217285 seconds.
eval score on batch: 0.20059990673472022
epsilons agent1 and agent2: 1 1


([(1, 1)],
 [1.7375359535217285],
 16,
 13,
 [0.20059990673472022],
 [0.7785100936889648])

In [76]:
outputs = []

In [79]:
nn_2 = cnn_model()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 2, 2, 64)          1216      
_________________________________________________________________
flatten_16 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_47 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 129       
Total params: 100,033
Trainable params: 100,033
Non-trainable params: 0
_________________________________________________________________
None


In [94]:
agent1 = nn_agent(player = 1,  nn = nn_1, epsilon_min = .1, epsilon = .3, epsilon_decay = .995)
agent2 = nn_agent(player = -1,  nn = nn_2, epsilon_min = .1, epsilon = .3, epsilon_decay = .995)

In [81]:
time_str = str(datetime.now())[:19].replace(':','_')
with open(f'outputs_'+time_str+'.pickle', 'wb') as file:
    pickle.dump(outputs, file, protocol = pickle.HIGHEST_PROTOCOL)

In [97]:
##################################################
## Change number of games to simulate here
#_games = 10000
##################################################
time0 = time.time()
for x in range(27):
    outputs.append(big_sim_parallel_nn(agent1, agent2, n_steps=20, games_per_step=32, nn_to_update=[nn_1, nn_2], parallel_threads=6))
    print(len(outputs), "outputs loaded.")
print(time.time()-time0, 'seconds.')

760/760 [==============================] - 0s 48us/step
updating with 760, 760 training batch.
Epoch 1/1
760/760 [==============================] - 0s 202us/step - loss: 0.2001
Epoch 1/1
760/760 [==============================] - 0s 201us/step - loss: 0.1998
player 1 wins: 19
player 2 wins: 11
simulations took 53.884236097335815 seconds.
neural network update time: 0.31261706352233887 seconds.
eval score on batch: 0.20240403511806537
epsilons agent1 and agent2: 0.26599605315039226 0.26599605315039226
792/792 [==============================] - 0s 37us/step
updating with 792, 792 training batch.
Epoch 1/1
792/792 [==============================] - 0s 154us/step - loss: 0.1903
Epoch 1/1
792/792 [==============================] - 0s 153us/step - loss: 0.1905
player 1 wins: 19
player 2 wins: 11
simulations took 56.347805976867676 seconds.
neural network update time: 0.24923396110534668 seconds.
eval score on batch: 0.19175122256832894
epsilons agent1 and agent2: 0.2646660728846403 0.2646660

In [99]:
random_agent = nn_agent(player = -1,  nn = nn_1, epsilon_min = 0, epsilon = 1, epsilon_decay = 1)  # no decay, fully random
agent1.epsilon = 0 # set agent1 to no random moves
returns3 = big_sim_parallel_nn(agent1, random_agent, n_steps=1, games_per_step=32, nn_to_update=[], parallel_threads=6) # no update

game_step 0 updating with 666, 666 training batch.
player 1 wins: 26
player 2 wins: 6
simulations took 54.458451986312866 seconds.
neural network update time: 0.00013375282287597656 seconds.


IndexError: list index out of range

In [23]:
59738/60/60


16.593888888888888

In [14]:
len(qtable1.q_dict)
#agent1.epsilon

5444204

x = [2,2,2,2,2]

with open('test.pickle', 'wb') as file:
    pickle.dump(x, file, protocol = pickle.HIGHEST_PROTOCOL)
    
with open('test.pickle', 'rb') as file:
    y = pickle.load(file)

print(y)

In [15]:
ns = []
for k,v in qtable1.q_dict.items():
    ns.append(v[1])


In [16]:
np.histogram(np.array(ns))

(array([5444199,       3,       1,       0,       0,       0,       0,
              0,       0,       1]),
 array([1.000000e+00, 1.797250e+04, 3.594400e+04, 5.391550e+04,
        7.188700e+04, 8.985850e+04, 1.078300e+05, 1.258015e+05,
        1.437730e+05, 1.617445e+05, 1.797160e+05]))

In [17]:
print(len([x for x in ns if x != 1]))
print(len([x for x in ns if x > 2]))
print(len([x for x in ns if x > 3]))
print(len([x for x in ns if x > 4]))
print(len([x for x in ns if x > 10]))

153873
67959
46602
33784
12230


In [18]:
i = 0
for k,v in qtable1.q_dict.items():
    print(k,v)
    i += 1
    if i > 5: break

[120020100111022211102201112221201221] (0.5, 1)
[020011100211022012102201112221201221] (0.5, 1)
[002011202211010012102201112201201221] (0.5, 1)
[002011202211010012210201010201122221] (0.5, 1)
[020011100211022012210001010201122221] (0.5, 1)
[110020112001200220022012002010111221] (0.5, 1)


In [19]:
np.histogram([x for x in ns if x != 1])

(array([94371,     6,     0,     0,     0,     0,     0,     0,     0,
            1]),
 array([2.00000e+00, 1.15380e+04, 2.30740e+04, 3.46100e+04, 4.61460e+04,
        5.76820e+04, 6.92180e+04, 8.07540e+04, 9.22900e+04, 1.03826e+05,
        1.15362e+05]))